Load cleaned order data into Databricks

In [0]:
orders_df = spark.read.csv("/FileStore/tables/orders.csv", header=True, inferSchema=True)
display(orders_df)


order_id,customer_id,order_date,delivery_date,status_description
1,6,2025-07-19,2025-07-23,Courier not assigned
2,4,2025-07-18,2025-07-24,Wrong address
3,7,2025-07-20,2025-07-23,On time
4,1,2025-07-19,2025-07-28,Wrong address
5,7,2025-07-12,2025-07-18,Delayed due to rain
6,1,2025-07-13,2025-07-17,Wrong address
7,3,2025-07-12,2025-07-17,Courier not assigned
8,9,2025-07-11,2025-07-15,Delayed due to rain
9,4,2025-07-16,2025-07-24,Delayed due to rain
10,10,2025-07-14,2025-07-22,Packaging delay


Create a pipeline to update latest delivery status

In [0]:
from pyspark.sql.functions import current_date

# Add a column "last_updated" to simulate ETL update
updated_orders = orders_df.withColumn("last_updated", current_date())

display(updated_orders.limit(5))


order_id,customer_id,order_date,delivery_date,status_description,last_updated
1,6,2025-07-19,2025-07-23,Courier not assigned,2025-08-17
2,4,2025-07-18,2025-07-24,Wrong address,2025-08-17
3,7,2025-07-20,2025-07-23,On time,2025-08-17
4,1,2025-07-19,2025-07-28,Wrong address,2025-08-17
5,7,2025-07-12,2025-07-18,Delayed due to rain,2025-08-17


Save the results as Delta or CSV

In [0]:
updated_orders.write.format("delta").mode("overwrite").save("/mnt/delta/orders_cleaned")


Run SQL query to show top 5 delayed customers

In [0]:
# Register DataFrame as SQL view
updated_orders.createOrReplaceTempView("orders")

# Run SQL to find top 5 delayed customers
top_delayed = spark.sql("""
    SELECT customer_id, COUNT(*) AS delay_count
    FROM orders
    WHERE status_description <> 'On time'
    GROUP BY customer_id
    ORDER BY delay_count DESC
    LIMIT 5
""")
display(top_delayed)


customer_id,delay_count
10,4
4,3
9,3
3,2
1,2
